In [41]:
import tensorflow as tf
import datetime
import numpy as np
import tensorflow.keras.backend as K

In [19]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [20]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [64]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])


def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def true_predict(y_true, y_pred):
    result = 0
    print(y_pred)
    print(y_true)
#     for i in range(y_pred.shape):
#         if y_pred[i] == y_true[i]:
#             result += 1
#     return result

In [65]:
model = create_model()

In [66]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', mean_pred])


In [7]:
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# model.fit(x=x_train, 
#           y=y_train, 
#           epochs=5, 
#           validation_data=(x_test, y_test), 
#           callbacks=[tensorboard_callback])


In [8]:
#%tensorboard --logdir logs/fit --bind_all

In [67]:

batch_size = 10000
nb_epochs = 1
n_steps = int(len(x_train)/batch_size) #+1
epoch_scores = list()
epoch_scores_valid = list()

val_split = 0.1

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# before training init writer (for tensorboard log) / model
writer = tf.summary.create_file_writer(log_dir+'/train')
writer_valid = tf.summary.create_file_writer(log_dir+'/valid')


for epoch in range(nb_epochs):
    print('start epoch', epoch)
    for i in range(0, len(x_train), batch_size):

        start = i
        end_train = int(i+(batch_size*(1 - val_split)))
        end = i + batch_size - 1

        x_batch = x_train[start:end_train]
        y_batch = y_train[start:end_train]

        x_batch_valid = x_train[end_train:end]
        y_batch_valid = y_train[end_train:end]

        tmp = model.train_on_batch(x_batch, y_batch, reset_metrics=False)
        print(tmp)

        epoch_scores = model.evaluate(x_batch, y_batch)
        epoch_scores_valid = model.evaluate(x_batch_valid, y_batch_valid)

        with writer.as_default():
            tf.summary.scalar(name="loss", data=epoch_scores[0], step=i)
            tf.summary.scalar(name="accuracy", data=epoch_scores[1], step=i)
            tf.summary.scalar(name="mean_pred", data=epoch_scores[2], step=i)
            writer.flush()

        with writer_valid.as_default():
            tf.summary.scalar(name="loss", data=epoch_scores_valid[0], step=i)
            tf.summary.scalar(name="accuracy", data=epoch_scores_valid[1], step=i)
            tf.summary.scalar(name="mean_pred", data=epoch_scores_valid[2], step=i)
            writer_valid.flush()
            
            
            
    

    

start epoch 0
[2.3905091, 0.08188889, 0.1]
999/999 [==============================] - 0s 54us/sample - loss: 2.0348 - accuracy: 0.4434 - mean_pred: 0.1000
[2.08571, 0.3278328, 0.09999997]
999/999 [==============================] - 0s 52us/sample - loss: 1.8007 - accuracy: 0.5926 - mean_pred: 0.1000
[1.8360054, 0.5408541, 0.09999997]
999/999 [==============================] - 0s 59us/sample - loss: 1.6364 - accuracy: 0.6737 - mean_pred: 0.1000
[1.6397318, 0.63756377, 0.09999997]
999/999 [==============================] - 0s 51us/sample - loss: 1.4129 - accuracy: 0.7337 - mean_pred: 0.1000
[1.4280629, 0.7120712, 0.09999997]
999/999 [==============================] - 0s 52us/sample - loss: 1.3162 - accuracy: 0.7357 - mean_pred: 0.1000
[1.2238811, 0.7711771, 0.09999997]
999/999 [==============================] - 0s 63us/sample - loss: 0.9575 - accuracy: 0.8619 - mean_pred: 0.1000


In [10]:
oui = np.mean(epoch_scores, axis=0)
print(oui)

0.6044667162685164
